In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Lambda

# 1️⃣ Simulación de datos
np.random.seed(42)

# Variables numéricas
X_num = np.random.rand(1000, 3) * 10  # 3 columnas numéricas

# Variables categóricas
X_cat = np.random.choice(["A", "B", "C"], size=(1000, 2))  # 2 columnas categóricas

# Variable objetivo (multiclase con 4 categorías)
y = np.random.choice(["Clase1", "Clase2", "Clase3", "Clase4"], size=1000)

# Creación del DataFrame
columns_num = ["num1", "num2", "num3"]
columns_cat = ["cat1", "cat2"]
df = pd.DataFrame(np.hstack((X_num, X_cat)), columns=columns_num + columns_cat)
df[columns_num] = df[columns_num].astype(float)  # Asegurar tipos numéricos

# Codificamos la variable respuesta (Label Encoding)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Convierte clases en 0, 1, 2, 3

# 2️⃣ Separación de datos
X_train, X_test, y_train, y_test = train_test_split(df, y_encoded, test_size=0.2, random_state=42)

# 3️⃣ Preprocesamiento con ColumnTransformer
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), columns_num),
    ("cat", OneHotEncoder(handle_unknown="ignore"), columns_cat)
])

# Aplicamos la transformación
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# 4️⃣ Creación de la red neuronal
model = keras.Sequential([
    layers.Dense(32, activation="relu", input_shape=(X_train_transformed.shape[1],)),
    layers.Dense(16, activation="relu"),
    layers.Dense(4, activation="softmax")  # 4 clases → activación softmax
])

# 5️⃣ Compilación y entrenamiento
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train_transformed, y_train, epochs=30, batch_size=32, validation_data=(X_test_transformed, y_test))

# 6️⃣ Evaluación
test_loss, test_acc = model.evaluate(X_test_transformed, y_test)
print(f"Test Accuracy: {test_acc:.4f}")


Epoch 1/30


c:\Users\bruno\anaconda3\envs\TrainEnv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.2800 - loss: 1.3994 - val_accuracy: 0.2200 - val_loss: 1.4174
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2741 - loss: 1.4006 - val_accuracy: 0.2400 - val_loss: 1.4102
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2477 - loss: 1.3972 - val_accuracy: 0.2350 - val_loss: 1.4083
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2843 - loss: 1.3822 - val_accuracy: 0.2300 - val_loss: 1.4068
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2735 - loss: 1.3846 - val_accuracy: 0.2350 - val_loss: 1.4046
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2876 - loss: 1.3729 - val_accuracy: 0.2350 - val_loss: 1.4035
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2901 - loss: 1.3742 - val_accuracy: 0.2400 - val_loss: 1.4026
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3003 - loss: 1.3696 - val_accuracy: 0.2500 - val_loss: 1.4022
Ep

In [26]:
# Exportación Onnx
import onnx
from onnx import helper, TensorProto
import onnxruntime as rt
from skl2onnx import convert_sklearn
from onnxmltools.convert.common.data_types import FloatTensorType, StringTensorType, Int64TensorType

def convert_dataframe_schema(df,drop = None):
    inputs = []
    for k,v in zip(df.columns,df.dtypes):
        if drop is not None and k in drop:
            continue
        if v == 'int64':
            t = Int64tensorType([None,1])
        elif v == 'float64':
            t = FloatTensorType([None,1])
        else:
            t = StringTensorType([None,1])
        inputs.append((k,t))
    return inputs

initial_type = convert_dataframe_schema(X_train)
onnx_preprocessor = convert_sklearn(preprocessor, initial_types=initial_type)
onnx_preprocessor_path = "preprocessor.onnx"
with open(onnx_preprocessor_path, "wb") as f:
    f.write(onnx_preprocessor.SerializeToString())

print(f"Preprocesador exportado correctamente a {onnx_preprocessor_path}")

Preprocesador exportado correctamente a preprocessor.onnx


In [27]:
# Cargar modelo en onnx model
import tf2onnx
model.output_names=['output']
input_signature = [tf.TensorSpec(shape=X_train_transformed.shape, dtype=tf.float32)]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=input_signature, opset=13)
onnx.save_model(onnx_model, "ModelRNN.onnx")

rewriter <function rewrite_constant_fold at 0x000002BF7AB1E320>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.


In [28]:
preprocessing_model = onnx.load('preprocessor.onnx')
rnn_model = onnx.load('ModelRNN.onnx')

preprocessing_output_name = preprocessing_model.graph.output[0].name
rnn_model.graph.input[0].name = preprocessing_output_name

assert(
    preprocessing_output_name == rnn_model.graph.input[0].name
), 'Nombres igualados'

for counter, node in enumerate(rnn_model.graph.node):
    if counter == 0:
        node.input[0] = preprocessing_output_name
        break
combined_graph = onnx.helper.make_graph(
    nodes = list(preprocessing_model.graph.node) + list(rnn_model.graph.node),
    name = "UnifiedPipeline",
    inputs = preprocessing_model.graph.input,
    outputs = rnn_model.graph.output,
    initializer = list(preprocessing_model.graph.initializer)
    + list(rnn_model.graph.initializer),
)

unified_opset_version = 17

combined_opset_import = [
    onnx.helper.make_opsetid('ai.onnx.ml',1),
    onnx.helper.make_opsetid('',unified_opset_version)
]

combined_model = onnx.helper.make_model(
    combined_graph, opset_imports=combined_opset_import, ir_version = 9
)

onnx.save(combined_model,'ModeloUnido.onnx')

In [29]:
from onnx import numpy_helper

output_name = "output"  # Este es el nombre del tensor de salida original del modelo

# Crear el nodo ArgMax
argmax_node = helper.make_node(
    'ArgMax',  # Tipo de nodo
    inputs=[output_name],  # La entrada es la salida del modelo
    outputs=['argmax_output'],  # El nombre de la salida de ArgMax
    axis=1,  # A lo largo de las columnas (eje 1) para obtener el índice de la clase
    keepdims=0  # No mantener las dimensiones adicionales
)

# Agregar el nodo ArgMax al grafo del modelo
combined_model.graph.node.append(argmax_node)

# Asegúrate de agregar la nueva salida al modelo
combined_model.graph.output[0].name = 'argmax_output'

# Guardar el modelo modificado
onnx.save(onnx_model, "modelo_con_argmax.onnx")